In [ ]:
"""
This notebook was me playing around with edge detectors to see if they would work for this problem. Didn't really work that well.
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def apply_canny(image):
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Canny(gray_frame, 100, 200)

def apply_sobel(image):
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobel_x = cv2.Sobel(gray_frame, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray_frame, cv2.CV_64F, 0, 1, ksize=3)

    abs_sobel_x = cv2.convertScaleAbs(sobel_x)
    abs_sobel_y = cv2.convertScaleAbs(sobel_y)

    combined_sobel = cv2.addWeighted(abs_sobel_x, 0.5, abs_sobel_y, 0.5, 0)
    return combined_sobel

def apply_scharr(image):
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    scharr_x = cv2.Scharr(gray_frame, cv2.CV_64F, 1, 0)
    scharr_y = cv2.Scharr(gray_frame, cv2.CV_64F, 0, 1)

    abs_scharr_x = cv2.convertScaleAbs(scharr_x)
    abs_scharr_y = cv2.convertScaleAbs(scharr_y)

    combined_scharr = cv2.addWeighted(abs_scharr_x, 0.5, abs_scharr_y, 0.5, 0)
    return combined_scharr

In [ ]:
video_path = "data/20231017_120545.mp4"

video = cv2.VideoCapture(video_path)

In [ ]:
status, frame = video.read()
# cv2.imshow("Original", frame)
if status:
    canny = apply_canny(frame)
    cv2.imshow("Canny Result", canny)
    cv2.imwrite("canny.jpg", canny)
    
    sobel = apply_sobel(frame)
    # cv2.imshow("Sobel Result", sobel)
    
    scharr = apply_scharr(frame)
    # cv2.imshow("Scharr Result", scharr)

# while True:
#     status, frame = video.read()
    
#     if status:
#         canny = apply_canny(frame)
#         sobel = apply_sobel(frame)
#         scharr = apply_scharr(frame)
    
#     cv2.imshow("Canny Result", canny) # Complete loss during microtome movement
#     cv2.imshow("Sobel Result", sobel) # Noisy
#     cv2.imshow("Scharr Result", scharr) # Noisy
    
#     key = cv2.waitKey(40)
#     if key & 0xFF == ord('q'):
#         break



In [ ]:
roi_points = []
selecting_roi = False

# Mouse callback function to give a bit of a helping hand for where to look for slices.
def select_roi(event, x, y, flags, param):
    global roi_points, selecting_roi

    # Left mouse button down
    if event == cv2.EVENT_LBUTTONDOWN:
        roi_points = [(x, y)]
        selecting_roi = True

    # Left mouse button up
    elif event == cv2.EVENT_LBUTTONUP:
        roi_points.append((x, y))
        selecting_roi = False

        # Draw rectangle on the image for visualization
        cv2.rectangle(img, roi_points[0], roi_points[1], (255,0,0), 2)
        cv2.imshow('Image', img)

# Load the image
img = cv2.imread('canny.jpg', cv2.IMREAD_GRAYSCALE)
cv2.imshow('Image', img)

# Set mouse callback
cv2.setMouseCallback('Image', select_roi)

# Wait for a key press and then process the ROI
cv2.waitKey(0)

# Extract and process the ROI after selecting
if len(roi_points) == 2:
    roi = img[roi_points[0][1]:roi_points[1][1], roi_points[0][0]:roi_points[1][0]]
    
    # Binary thresholding
    _, roi = cv2.threshold(roi, 127, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to attempt to close gaps
    kernel = np.ones((7,7), np.uint8)  # Adjust kernel for gap closing
    closing = cv2.morphologyEx(roi, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours in the image
    contours, _ = cv2.findContours(closing, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around detected trapezoids / contours
    for contour in contours:
        if cv2.contourArea(contour) > 100:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(roi, (x, y), (x + w, y + h), (255, 255, 255), 2)

    # Display the processed ROI
    cv2.imshow('Processed ROI', roi)
    cv2.waitKey(0)

cv2.destroyAllWindows()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\morph.dispatch.cpp:1163: error: (-215:Assertion failed) !_src.empty() in function 'cv::morphologyEx'


In [2]:
import cv2

video = cv2.VideoCapture("data/20231017_120545.mp4")

for i in range(5):
        
    _, frame = video.read()

    cv2.imwrite(f"data/frame_{i}.jpg", frame)
